<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso3/ciclo4/M6U4_Taller_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=1o4udU5qVMi_7jDi0XzSspbPC6Hw0ev9o" width="100%">

# **Taller 4: Despliegue de modelos**
---

En este notebook evaluaremos los conceptos aprendidos sobre el despliegue de modelos.

Ejecute las siguientes celdas para conectarse a UNCode:

In [ ]:
!pip install rlxcrypt

In [ ]:
!wget --no-cache -O session.pye -q https://raw.githubusercontent.com/JuezUN/INGInious/master/external%20libs/session.pye

In [ ]:
import rlxcrypt
import session

grader = session.LoginSequence("MAPEDDACML-GroupMLDS-6-2024-2@2907beb0-f808-419c-a736-2406605bf0ad")

Comenzamos instalando las librerías y herramientas necesarias:

In [ ]:
!pip install fastapi

Importamos las librerías necesarias:

In [ ]:
# Librerías de utilidad para manipulación y visualización de datos.
import joblib
import requests
import pandas as pd
import os
import subprocess
from sklearn.svm import SVR
from IPython.display import display
from pydantic import BaseModel
from typing import List

# Ignorar warnings.
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Versiones de las librerías usadas
!python --version
import fastapi
print('FastAPI', fastapi.__version__)

Esta actividad se realizó con las siguientes versiones:
*  Python 3.10.11
*  FastAPI 0.95.11

## **Carga de datos**
----

Se trata de un conjunto de datos que contiene valores monetarios de las acciones de la empresa ABC durante seis días consecutivos, las columnas están nombradas de forma relativa de acuerdo al día, donde el primer día corresponde a `d1`, el segundo a `d2`, y así con el resto de días.

<center><img src = "https://drive.google.com/uc?export=view&id=1_zfM8NlHwTjJlilr8hSC0MEXMFKyVqKl" alt = "Encabezado MLDS" width = "80%">  </img></center>

El objetivo es utilizar los 5 primeros días para predecir el valor de la acción durante el sexto día.

In [ ]:
data = pd.read_parquet("https://raw.githubusercontent.com/mindlab-unal/mlds6-datasets/main/u4/stocks.parquet")
display(data.head())

## **1. Selección de Datos**
---
En este punto deberá separar la columna objetivo de las variables que serán entrada para el modelo.

Para esto debe implementar la función `get_xy` la cual toma como entrada el conjunto de datos y debe retornar la matriz de características y el vector de etiquetas.

**Parámetros**

- `df`: conjunto de datos como un `pd.DataFrame`.

**Retorna**

- `features`: arreglo de `numpy` con las características.
- `labels`: arreglo de `numpy` con las etiquetas.

In [ ]:
# FUNCIÓN CALIFICADA get_xy
def get_xy(df):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    features, labels = ..., ...
    return features, labels
    ### FIN DEL CÓDIGO ###

Use las siguientes celdas para probar su solución:

In [ ]:
#TEST_CELL
features, labels = get_xy(data)
display(features.sum())

El resultado de la celda anterior debería ser:

```python
❱ display(features.sum())
34949.00878930474
```

In [ ]:
#TEST_CELL
features, labels = get_xy(data)
display(features.shape)
display(labels.shape)

El resultado de la celda anterior debería dar los tamaños esperados de los arreglos:

```python
❱ display(features.shape)
(1000, 5)

❱ display(labels.shape)
(1000,)
```

<details>
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

* Recuerde convertir la matriz de características y el vector de etiquetas a arreglos de `numpy`.
</details>

<details>
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

* Puede usar el método `drop` para excluir una columna del `DataFrame`.
</details>

### **Evaluar código**

In [ ]:
grader.run_test("Test 1_1", globals())

## **2. Creación del Repositorio**
---

En este punto deberá crear un repositorio de `git` de la siguiente forma:

1. El nombre del repositorio debe ser `stocks`.
2. Debe ubicarse dentro del repositorio e inicializarlo con `git`.

Para esto deberá autenticarse con `git`:

In [ ]:
!git config --global user.email "ejemplo@unal.edu.co"
!git config --global user.name "Mi nombre o username"
!git config --global init.defaultBranch master

In [ ]:
#TEST_CELL
"""
    Descomente esta celda si lo requiere
    Este comando borrará el directorio stocks
    si existe en el directorio actual
"""
# ![ -d stocks ] && rm -rf stocks

In [ ]:
# INGRESE SU CÓDIGO AQUÍ

Use las siguientes celdas para probar su solución:

In [ ]:
#TEST_CELL
!pwd | awk -F '/' '{print $NF}'

**Salida esperada**

En este caso debería obtener el nombre del repositorio:

```python
❱ !pwd | awk -F '/' '{print $NF}'
stocks
```

In [ ]:
#TEST_CELL
!git status

**Salida esperada**

En este caso se valida que estemos dentro de un repositorio de `git`.

```python
❱ !git status
On branch main

No commits yet

nothing to commit (create/copy files and use "git add" to track)
```

<details>
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

* Recuerde que con `mkdir` puede crear carpetas.
</details>

<details>
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

* El comando `%cd` es necesario para movimientos dentro de linux.
</details>

<details>
<summary>
    <font size="3" color="darkgreen"><b>Pista 3</b></font>
</summary>

* Recuerde que dentro de un terminal, el comando `git` le permite realizar cualquier operación con un repositorio.
</details>

### **Evaluar código**

> **Esta celda debe ser ejecutada obligatoriamente antes de evaluar el ejercicio**

In [ ]:
result = subprocess.run(['pwd'], stdout=subprocess.PIPE)
current_directory = result.stdout.decode('utf-8').strip().split('/')[-1]

In [ ]:
grader.run_test("Test 2_1", globals())

## **3. Modelamiento**
---

En este punto deberá entrenar un modelo de máquina de soporte vectorial para regresión a partir de los datos.

Para esto debe implementar la función `train_model` la cual toma como entrada la matriz de características y el vector de etiquetas y debe retornar el modelo entrenado.

**Parámetros**

- `features`: arreglo de `numpy` con las características.
- `labels`: arreglo de `numpy` con las etiquetas.

**Retorna**

- `model`: modelo entrenado.

In [ ]:
# FUNCIÓN CALIFICADA train_model
def train_model(features, labels):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    model = ...
    return model
    ### FIN DEL CÓDIGO ###

Use las siguientes celdas para probar su solución:

In [ ]:
#TEST_CELL
features, labels = get_xy(data)
model = train_model(features, labels)
print(model.score(features, labels))

El resultado de la celda anterior debería ser:

```python
❱ print(model.score(features, labels))
0.9804584510787114
```

<details>
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

* Recuerde que la clase `SVR` le permite definir una máquina de soporte vectorial para regresión.
</details>

<details>
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

* Tenga en cuenta que el modelo debe ser entrenado antes de retornarlo.
</details>

### **Evaluar código**

In [ ]:
grader.run_test("Test 3_1", globals())

## **4. Persistencia de Modelo**
---

En este punto deberá escribir una función que permita guardar un modelo entrenado con el nombre `model.joblib`.

Para esto, debe implementar la función `save_model` la cual toma como entrada un modelo entrenado y la ruta de almacenamiento del archivo para guardarlo.

**Parámetros**

- `model`: modelo entrenado.
- `path`: ruta donde se guardará el modelo.

In [ ]:
# FUNCIÓN CALIFICADA save_model
def save_model(model, path):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    ...
    ### FIN DEL CÓDIGO ###

Use las siguientes celdas para probar su solución:

In [ ]:
#TEST_CELL
save_model(model, "model.joblib")
!ls -sh

El resultado de la celda anterior debería ser:

```python
❱ !ls -sh
total 28K
28K model.joblib
```

<details>
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

* Recuerde que puede usar la librería `joblib` para guardar modelos de `sklearn`.
</details>

<details>
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

* Revise el funcionamiento de la función `joblib.dump`.
</details>

### **Evaluar código**

> **Esta celda debe ser ejecutada obligatoriamente antes de evaluar el ejercicio**

In [ ]:
save_model(model, "model.joblib")
comm = subprocess.check_output(['ls', '-sh'])
saved_directory = (comm.decode('utf-8')).split()

In [ ]:
grader.run_test("Test 4_1", globals())

## **5. Estructuración de API**
---

En este punto deberá escribir una clase que representará la entrada y la salida de un API para el modelo en `fastapi`.

Para esto debe implementar las clases `ApiInput` y `ApiOutput` con las siguientes consideranciones:

- `ApiInput`: debe tener un atributo llamado `features` que tenga tipo `List[float]`.
- `ApiOutput`: debe tener un atributo llamado `forecast` que sea de tipo `float`.

In [ ]:
# CLASE CALIFICADA ApiInput
class ApiInput(...):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    ...
    ### FIN DEL CÓDIGO ###

In [ ]:
# CLASE CALIFICADA ApiOutput
class ApiOutput(...):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    ...
    ### FIN DEL CÓDIGO ###

Use las siguientes celdas para probar su solución:

In [ ]:
#TEST_CELL
inp = ApiInput(features=[1.1, 2.2, 3.3])
display(inp)

El resultado de la celda anterior debería ser:

```python
❱ display(inp)
ApiInput(features=[1.1, 2.2, 3.3])
```

In [ ]:
#TEST_CELL
out = ApiOutput(forecast=4.5)
display(out)

El resultado de la celda anterior debería ser:

```python
❱ display(out)
ApiOutput(forecast=4.5)
```

<details>
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

* Las clases que estructuran las entradas del API deberán heredar de `BaseModel`.
</details>

<details>
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

* Recuerde utilizar la notación `:` para especificar anotaciones de tipo.
</details>

### **Evaluar código**

> **Esta celda debe ser ejecutada obligatoriamente antes de evaluar el ejercicio**

In [ ]:
"""
    Esta celda debe ser ejecutada obligatoriamente antes de evaluar el código.
    No la debe modificar
"""

def get_api_input(features: List[float]) -> ApiInput:
    return ApiInput(features=features)

def get_api_output(forecast: float) -> ApiOutput:
    return ApiOutput(forecast=forecast)

In [ ]:
grader.run_test("Test 5_1", globals())

## **6. Función del API**
---

En este punto deberá implementar una función que a partir de la clase de entrada, genere la clase de salida por medio del modelo.

Para esto debe utilizar la función `predict`, la cual toma como entrada un `ApiInput` y debe generar un `ApiOutput` al cargar y utilizar el modelo guardado en la ruta `model.joblib`.

**Parámetros**

- `data`: un objeto de tipo `ApiInput`.

**Retorna**

- `prediction`: un objeto de tipo `ApiOutput`.

In [ ]:
# FUNCIÓN CALIFICADA train_model
def predict(data: ApiInput) -> ApiOutput:
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    prediction = ...
    return prediction
    ### FIN DEL CÓDIGO ###

Use las siguientes celdas para probar su solución:

In [ ]:
#TEST_CELL
inp = ApiInput(features=[1.1, 2.2, 3.3, 3.2, 0.9])
print(type(inp), inp.features)

pred = predict(inp)
display(pred)

In [ ]:
inp = ApiInput(features=[52.6, 5.4, 1.85, 9.62, 3.2])
pred = predict(inp)
display(pred)

El resultado de la celda anterior debería ser:

```python
❱ display(pred)
ApiOutput(forecast=9.756829361821138)
```

### **Evaluar código**

In [ ]:
grader.run_test("Test 6_1", globals())

## **7. Despliegue del modelo (OPCIONAL)**
---

En este punto deberá seguir los siguientes pasos:

1. Completar el script `main.py`
2. Subir todo a un repositorio de github y desplegar el API.

Debe completar el siguiente script:

> **Nota**: `Railway` solo permite conectar cuentas de _Github_ que tengan una identidad confiable al tener cierta trayectoria, es por eso que si tienes una cuenta nueva no podrás desarrollar este punto en el momento

In [ ]:
%%writefile main.py
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List
import joblib

# Reemplace esto con su implementación:
class ApiInput(...):
    ...

# Reemplace esto con su implementación:
class ApiOutput(...):
    ...

app = FastAPI()
model = joblib.load("model.joblib")

# Reemplace esto con su implementación:
@app.post("/predict")
async def predict(data: ApiInput) -> ApiOutput:
    ...

<details>
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

* Recuerde que puede utilizar el procedimiento visto en el notebook `3_fastapi`.
</details>

Utilice los siguientes archivos de configuración:

In [ ]:
%%writefile requirements.txt
scikit-learn==1.2.2
fastapi==0.82.0
uvicorn==0.19.0

In [ ]:
%%writefile railway.json
{
  "$schema": "https://railway.app/railway.schema.json",
  "build": {
    "builder": "NIXPACKS"
  },
  "deploy": {
    "startCommand": "uvicorn main:app --host 0.0.0.0 --port $PORT",
    "restartPolicyType": "ON_FAILURE",
    "restartPolicyMaxRetries": 10
  }
}

Escriba los comandos para subir todo al repositorio de **GitHub**:

In [ ]:
# INGRESE SU CÓDIGO AQUÍ

Si el despliegue fue correcto, debe pegar la url del modelo en la siguiente variable:

In [ ]:
# INGRESE SU URL AQUÍ
model_url = "https://mlapi-production.up.railway.app"

In [ ]:
#TEST_CELL
inp = ApiInput(features=[1.1, 2.2, 3.3, 3.2, 0.9])
r = requests.post(
    os.path.join(model_url, "predict"),
    json=inp.dict(),
    )
print(r.json())

El resultado de la celda anterior debería ser:

```python
{"forecast": 9.756829361821138}
```

# **Evaluación**

In [ ]:
grader.submit_task(globals())

# **Créditos**
---

* **Profesor:** [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/).

* **Asistentes docentes:** [Juan Sebastián Lara Ramírez](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/).
* **Diseño de imágenes:**
  - [Rosa Alejandra Superlano Esquibel](https://www.linkedin.com/in/alejandra-superlano-02b74313a/).
  - [Mario Andrés Rodríguez Triana](mailto:mrodrigueztr@unal.edu.co).

* **Coordinador de virtualización:** [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*